In [1]:
####FINAL PROJECT RAG CHATBOT APPLICATION#####
import pandas as pd
# Load the parquet file containing the European history data
data_path = "C:/Users/lucaz/Downloads/BIA6304_Final/EuropeanHistory.parquet"
df = pd.read_parquet(data_path)

# View the dataset
df.head()

,category,subcategory,title,content,source
0,European History,European Prehistory,Neolithic Revolution,"The Neolithic Revolution, also known as the Fi...",https://en.wikipedia.org/wiki/Neolithic_Revolu...
1,European History,European Prehistory,Ötzi The Iceman,"Ötzi, also called The Iceman, is the natural m...",https://en.wikipedia.org/wiki/Ötzi
2,European History,European Prehistory,Iron Age Societies,"In Europe, the Iron Age is the last stage of t...",https://en.wikipedia.org/wiki/Iron_Age_Europe
3,European History,European Prehistory,Balkan Neolithic Agriculture,The European Neolithic is the period from the ...,https://en.wikipedia.org/wiki/Neolithic_Europe
4,European History,European Prehistory,Ice Age Art,"In archaeology, cave paintings are a type of p...",https://en.wikipedia.org/wiki/Cave_painting


In [2]:
from embedding import embedding_model
from search import search

# Example: Query for similarity search
query_text = "Roman culture."
query_embedding = embedding_model(query_text)

# Call the search function and get results
results_df = search(query_text, top_k=3)

# Print the results
print("Search Results:")
for index, row in results_df.iterrows():
    print(f"ID: {row['id']}")
    print(f"Title: {row['title']}")
    print(f"Content: {row['content']}")
    print(f"Similarity Score: {row['similarity_score']}\n")

Search Results:
ID: 24
Title: Roman Republic
Content: The Roman Republic (Latin: Res publica Romana) was the era of classical Roman civilization beginning with the overthrow of the Roman Kingdom (traditionally dated to 509 BC) and ending in 27 BC with the establishment of the Roman Empire following the War of Actium. During this period, Rome's control expanded from the city's immediate surroundings to hegemony over the entire Mediterranean world. Roman society at the time was primarily a cultural mix of Latin and Etruscan societies, as well as of Sabine, Oscan, and Greek cultural elements, which is especially visible in the Ancient Roman religion and its Pantheon. Its political organization developed at around the same time as direct democracy in Ancient Greece, with collective and annual magistracies, overseen by a senate. There were annual elections, but the republican system was an elective oligarchy, not a democracy; a small number of powerful families largely monopolised the magis

In [3]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

# Check if API key was loaded correctly
if api_key is None:
    raise ValueError("API key not found. Please check your .env file.")

print(f"API Key loaded successfully: {api_key[:5]}****")  # Only print part of the key for security

API Key loaded successfully: AIzaS****


In [8]:
import os
from dotenv import load_dotenv
from ChatConfig import ChatConfig  # Import the ChatConfig class
from chatbot import chatbot  # Import the chatbot function
from llm import AIStudioProvider

# Load the .env file
load_dotenv()

def main():
    # Access the API key
    api_key = os.getenv("API_KEY")
    
    if api_key is None:
        raise ValueError("API key not found. Please check your .env file.")

    # Create an instance of ChatConfig and load API key
    chat_config = ChatConfig()
    chat_config.load_api_key(api_key)  # Assuming load_api_key is a method of ChatConfig
    
    # Run the chatbot function
    chatbot(chat_config)



In [9]:
from dotenv import load_dotenv
from ChatConfig import ChatConfig  # Import the ChatConfig class
from llm import AIStudioProvider  # Ensure this is available
from strategy import ChatTurnStrategy
import logging
import pandas as pd

# Load the .env file
load_dotenv()
api_key = os.getenv("API_KEY")
logging.basicConfig(level=logging.DEBUG)

# Create an instance of ChatConfig
chat_config = ChatConfig()
chat_config.load_api_key(api_key)  # Load the API key into the config

# Mock search function for testing
def mock_search_function(user_input, top_k=3):
    # Create a DataFrame to simulate search results
    data = {
        "id": [1, 2],
        "title": [
            "The Beginning of AI",
            "Recent Advances in AI"
        ],
        "content": [
            "Artificial intelligence (AI) began as a field in 1956 at a conference at Dartmouth College.",
            "AI research has made significant progress in recent years, including advancements in machine learning."
        ],
        "similarity_score": [0.9, 0.85]  # Mock similarity scores
    }
    
    # Create the DataFrame
    return pd.DataFrame(data)

# Create an instance of ChatTurnStrategy with the mocked search function
chat_turn_strategy = ChatTurnStrategy(search_function=mock_search_function, chat_config=chat_config)  # Use chat_config instance

# Prepare your test inputs
user_input = "Tell me about the history of AI."

# Call the chat_turns_for method, simulating the conversation
response_history = chat_turn_strategy.chat_turns_for(user_input, history=[])

# Extract the assistant's response from the history
assistant_response = response_history[-1]["content"]  # Get the last entry, which is the assistant's turn
print("AI Response:", assistant_response)

AI Response: The field of Artificial Intelligence (AI) officially began in 1956 at a conference held at Dartmouth College. This gathering of leading researchers marked the birth of AI as a distinct discipline. 

Since then, AI research has gone through periods of both rapid advancement and slower progress. In recent years, we've seen a resurgence of interest and significant breakthroughs, particularly in the area of machine learning. This progress has led to the development of powerful AI systems capable of performing complex tasks, such as image recognition, language translation, and even playing games at a superhuman level. 

While AI has come a long way, it's still a relatively young field with much potential for future development.


In [10]:
from llm import AIStudioProvider
from search import search
from strategy import ChatTurnStrategy
from ChatConfig import ChatConfig  # Ensure you import ChatConfig correctly

# Define ConversationState class
class ConversationState:
    def __init__(self):
        self.history = []
        self.context = ""  # You should define context if used in the chatbot

# Function to run the chatbot loop
def chatbot(chat_config):
    print("Chatbot started. Type 'exit' to quit.")

    # Use the AIStudioProvider with the api_key loaded in chat_config
    ai_provider = AIStudioProvider(api_key=chat_config.api_key)

    if ai_provider:
        print("AIStudioProvider initialized successfully.")

    # Create the ChatTurnStrategy instance, passing the search function and chat configuration
    strategy = ChatTurnStrategy(search_function=search, chat_config=chat_config)

    # Initialize conversation state to track history and context
    state = ConversationState()

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        
        # Use the strategy to handle turns, providing history and context
        state.history = strategy.chat_turns_for(user_input, state.history, state.context)
        
        # Display user input and bot response from the conversation history
        for turn in state.history[-2:]:  # Show last two turns (user and assistant)
            print(f"{turn['role'].capitalize()}: {turn['content']}")


In [15]:
if __name__ == "__main__":
    main() 

Chatbot started. Type 'exit' to quit.
AIStudioProvider initialized successfully.
User: hello
Assistant: Hello! How can I assist you today?
User: Can you tell me something about the Roman culture?
Assistant: Roman culture was a fascinating blend of different influences, evolving over centuries. It started as a mix of Latin and Etruscan traditions, with elements from Sabine, Oscan, and Greek cultures. This diverse heritage is evident in their religion and pantheon of gods. 

The Roman Republic was a period of expansion and conquest, where Rome grew from a small city to a dominant force in the Mediterranean. This period was marked by constant warfare, with Rome facing off against various enemies like the Gauls, Carthaginians, and various kings and tribes. While the Republic was not a democracy, it had a complex political system with annual elections and a powerful senate.

The Pax Romana, a period of peace and prosperity under the Roman Empire, brought about a significant expansion of Rom